In [9]:
import rospy
import ipywidgets as widgets
import actionlib
from actionlib import GoalStatus
from assignment2_rt.msg import PlanningAction, PlanningGoal
from assignment2_rt.msg import Robot_info
from nav_msgs.msg import Odometry

In [10]:
pub_position_vel = None # Global variable that contains the publisher information.

In [11]:
def odom_callback(msg):        
    robot_info = Robot_info()
    robot_info.x = msg.pose.pose.position.x
    robot_info.y = msg.pose.pose.position.y
    robot_info.vel_x = msg.twist.twist.linear.x
    robot_info.vel_z = msg.twist.twist.angular.z

    pub_position_vel.publish(robot_info)

In [23]:
rospy.init_node('action_client')

pub_position_vel = rospy.Publisher('/robot_information', Robot_info, queue_size=10)
rospy.wait_for_message('/odom', Odometry)
rospy.Subscriber('/odom', Odometry, odom_callback)
    
client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
client.wait_for_server()

out_send, out_other = widgets.Output(), widgets.Output()
ui, x_widget, y_widget = None, None, None

def feedback_callback(feedback):
    global out_other
    with out_other:
        if (feedback.stat == "Target reached!"):
            print("")
            print("Target reached\n{}\nStatus: {}\n".format(feedback.actual_pose, feedback.stat))


def send_goal(client, x, y):        
    goal = PlanningGoal()
    goal.target_pose.pose.position.x = x
    goal.target_pose.pose.position.y = y
    
    goal.target_pose.pose.position.z = 0.0      
    goal.target_pose.pose.orientation.x = 0.0  
    goal.target_pose.pose.orientation.y = 0.0  
    goal.target_pose.pose.orientation.z = 0.0   
    goal.target_pose.pose.orientation.w = 0.0   

    client.send_goal(goal, feedback_cb=feedback_callback)
    print("Goal sent")

def cancel_goal(client):
    global out_other
    with out_other:
        if client.get_state() in [GoalStatus.ACTIVE, GoalStatus.PENDING]:
            print("Cancelling current goal")
            client.cancel_goal()
            rospy.sleep(0.5)  # Timeout for forward the cancel
            state = client.get_state()
            if state in [GoalStatus.PREEMPTED, GoalStatus.RECALLED]:
                print("Goal successfully cancelled")
            else:
                print("Failed to cancel the goal")
        else:
            print("No active goal to cancel.")
  
def confirm_clicked(b):
    global out_send, out_other, x_widget, y_widget
    
    if out_send or out_other is not None:
            out_other.clear_output()
            out_send.clear_output()
            
    with out_send:
        x, y = x_widget.value, y_widget.value
        if x is None or y is None:
            print("Inserisci valori validi per X e Y")
            return
        send_goal(client, x, y)
        print(f"Goal inviato a ({x:.2f}, {y:.2f})")
        
def ask_input():
    global out_send, x_widget, y_widget, ui
    
    x_widget = widgets.BoundedFloatText(value=0.0, min=-20.0, max=20.0, step=1, description='X:')
    y_widget = widgets.BoundedFloatText(value=0.0, min=-20.0, max=20.0, step=1, description='Y:')
    
    confirm_button = widgets.Button(description='Invia coordinate')   

    ui = widgets.VBox([x_widget, y_widget, confirm_button, out_send])
    display(ui)
    
    confirm_button.on_click(confirm_clicked)
    
command = widgets.Dropdown(
    options=[('Select a command', 'n'),
            ('Set a new goal (s)', 's'),
             ('Cancel the current goal (c)', 'c'),
             ('Quit the action client (q)', 'q')],
    value='n',
    description='Command:',
)

display(command)
display(out_other)

def command_change(change):
    
    cmd = change['new']
    if cmd == 'n':
        if out_send or out_other is not None:
            out_other.clear_output()
            out_send.clear_output()
        if ui is not None:
            ui.layout.display = 'none'
    elif cmd == 's':
        if out_send or out_other is not None:
            out_other.clear_output()
            out_send.clear_output()
        ask_input()
    elif cmd == 'c':
        if out_other is not None:
            out_other.clear_output()
        if ui is not None:
            ui.layout.display = 'none'
        cancel_goal(client)
    elif cmd == 'q':
        command.disabled = True
        rospy.signal_shutdown("Quit from widget")
        print("Quit from widget")

command.observe(command_change, names='value')

ROSInterruptException: rospy shutdown